In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from inspect import getfullargspec

In [2]:
def compute_gradient(u,x0,dx=1e-7): #функция, приближенно вычисляющая градиент в точке
    n = len(x0)
    gradient = np.zeros(n)
    for i in range(n):
        delta = np.array([0 if j != i else dx for j in range(n)])
        gradient[i] = (u(*(x0+delta))-u(*x0))/dx
    return gradient

In [3]:
def gradient_descent(f,learning_rate,eps,dx=1e-7): #алгоритм градиентного спуска
    max_iters = 2000
    num_of_iters = 0
    n = len(getfullargspec(f)[0])
    teta0 = np.array([np.random.normal() for _ in range(n)])
    teta1 = teta0 - learning_rate*compute_gradient(f,teta0,dx)
    while(norm(teta0-teta1)>eps and num_of_iters < max_iters):
        num_of_iters +=1
        teta0 = teta1
        teta1 = teta0 - learning_rate*compute_gradient(f,teta0,dx)
    return teta0, num_of_iters

In [4]:
def nesterov_method(f,eps): #метод Нестерова
    n = len(getfullargspec(f)[0])
    y1 = np.array([np.random.normal() for _ in range(n)])
    z = np.array([np.random.normal() for _ in range(n)])
    k = 0
    a1 = 1
    x0 = y1
    a0 = norm(y1-z)/norm(compute_gradient(f,y1)-compute_gradient(f,z))
    i = 0
    while f(*y1)-f(*(y1-2**(-i)*a0*compute_gradient(f,y1)))<2**(-i-1)*a0*(norm(compute_gradient(f,y1))**2):
        i+=1
    a1 = 2**(-i)*a0
    x1 = y1-a1*compute_gradient(f,y1)
    a2 = (1+(4*a1**2+1)**0.5)/2
    y2 = x1+((a1-1)*(x1-x0))/a2
    while (norm(y1-y2)>eps and k < 2000):
        k+=1
        a0,a1,x0,y1 = a1,a2,x1,y2
        i = 0
        while f(*y1)-f(*(y1-2**(-i)*a0*compute_gradient(f,y1)))<2**(-i-1)*a0*(norm(compute_gradient(f,y1))**2):
            i+=1
        a1 = 2**(-i)*a0
        x1 = y1-a1*compute_gradient(f,y1)
        a2 = (1+(4*a1**2+1)**0.5)/2
        y2 = x1+((a1-1)*(x1-x0))/a2
    return y2,k

In [5]:
def g(x):
    return 4*x+11

In [6]:
N = 100
a,b = -10,10
h = (b-a)/N
X = np.array([a+i*h for i in range(N)])
Y = np.array([g(x) for x in X])

In [7]:
def loss_function(X,Y):
    return lambda a,b: np.sum(np.array([0.5*(a*x+b-y)**2 for x,y in zip (X,Y)]))

In [8]:
f = loss_function(X,Y)

In [9]:
ans_desc = pd.DataFrame(columns = ["lambda","eps","iters","|a_eps-a|","|b_eps-b|","loss"])

for i in range(2,7,2):
    for j in range(2,7,2):
        lmbda,eps = 10**(-i),10**(-j)
        res, k = gradient_descent(f,lmbda,eps)
        row = [lmbda,eps,k,abs(res[0]-4),abs(res[1]-11),f(*res)]
        ans_desc = ans_desc.append(pd.Series(row,index=ans_desc.columns),True)

ans_desc

,lambda,eps,iters,|a_eps-a|,|b_eps-b|,loss
0,0.010000,0.010000,6.0,4.380675e+09,4.103879e+08,3.198069e+22
1,0.010000,0.000100,7.0,5.567618e+09,8.388616e+11,3.518941e+25
2,0.010000,0.000001,6.0,3.580076e+09,9.843211e+03,2.136584e+22
3,0.000100,0.010000,250.0,3.071887e-03,9.934416e-01,4.933152e+01
4,0.000100,0.000100,698.0,3.075252e-05,9.929292e-03,4.928065e-03
5,0.000100,0.000001,1162.0,3.571931e-07,9.934884e-05,4.933674e-07
6,0.000001,0.010000,0.0,2.934287e+00,1.203571e+01,2.124269e+04
7,0.000001,0.000100,2000.0,3.819711e-02,1.032261e+01,5.326302e+03
8,0.000001,0.000001,2000.0,3.880933e-02,1.043288e+01,5.440710e+03


In [10]:
ans_nest = pd.DataFrame(columns = ["eps","iters","|a_eps-a|","|b_eps-b|","loss"])

for j in range(2,9,2):
    eps = 10**(-j)
    res, k = nesterov_method(f,eps)
    row = [eps,k,abs(res[0]-4),abs(res[1]-11),f(*res)]
    ans_nest = ans_nest.append(pd.Series(row,index=ans_nest.columns),True)

ans_nest

,eps,iters,|a_eps-a|,|b_eps-b|,loss
0,1.000000e-02,0.0,2.189094e+00,10.030417,1.279937e+04
1,1.000000e-04,490.0,4.150901e-04,0.134225,9.005510e-01
2,1.000000e-06,678.0,1.147943e-06,0.000355,6.302209e-06
3,1.000000e-08,1418.0,6.134034e-08,0.000004,6.771544e-10
